Imports

In [13]:
import sqlite3
import pandas as pd

Criando o banco e as tabelas

In [14]:
conn = sqlite3.connect('db_avaliacao_filmes.db')
cursor = conn.cursor()

In [15]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Filme (
id_filme INTEGER PRIMARY KEY,
nome_filme VARCHAR(100),
genero_filme VARCHAR(30),
imdb_link VARCHAR(1000),
img_link VARCHAR(1000),
popularidade_filme INT,
duracao_filme FLOAT,
sinopse VARCHAR(1000),
diretor_filme VARCHAR(50),
ano_lancamento INT,
nota_filme FLOAT,
produtora_filme VARCHAR(100)
)
''')

In [16]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Usuario (
id_usuario INTEGER PRIMARY KEY AUTOINCREMENT,
username VARCHAR(100)
)
''')

In [17]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Avaliacao (
id_avaliacao INTEGER PRIMARY KEY AUTOINCREMENT,
id_usuario INTEGER,
fk_filme_id INTEGER,
nota_avaliacao FLOAT,
FOREIGN KEY (id_usuario) REFERENCES Usuario (id_usuario),
FOREIGN KEY (fk_filme_id) REFERENCES Filme (id_filme)
)
''')

In [18]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Ator (
id_ator INT PRIMARY KEY,
nome_ator VARCHAR(100)
)
''')

In [19]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS filme_ator (
id_filme INT,
id_ator INT,
FOREIGN KEY (id_filme) REFERENCES Filme (id_filme),
FOREIGN KEY (id_ator) REFERENCES Ator (id_ator)
)
''')

In [20]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS idioma (
id_idioma INT PRIMARY KEY,
nome_idioma VARCHAR(100)
)
''')

In [21]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS filme_idioma (
id_filme INT,
id_idioma INT,
FOREIGN KEY (id_filme) REFERENCES Filme (id_filme),
FOREIGN KEY (id_idioma) REFERENCES idioma (id_idioma)
)
''')


conn.commit()

Populando o banco

In [22]:
filmes_df = pd.read_csv('filmes.csv')
filmes_df.to_sql('Filme', conn, if_exists='append', index=False)

atores_df = pd.read_csv('atores.csv')
atores_df.to_sql('Ator', conn, if_exists='append', index=False)

usuarios_df = pd.read_csv('usuarios.csv')
usuarios_df.to_sql('Usuario', conn, if_exists='append', index=False)

avaliacoes_df = pd.read_csv('avaliacoes.csv')
avaliacoes_df.to_sql('Avaliacao', conn, if_exists='append', index=False)

filme_ator_df = pd.read_csv('filme_ator.csv')
filme_ator_df.to_sql('filme_ator', conn, if_exists='append', index=False)

filme_idioma_df = pd.read_csv('filme_idioma.csv')
filme_idioma_df.to_sql('filme_idioma', conn, if_exists='append', index=False)

20

In [23]:
conn.commit()

Conectando com o banco e fazendo queries

In [24]:
conn = sqlite3.connect('db_avaliacao_filmes.db')

In [26]:
query = '''
SELECT U.id_usuario, U.username, F.id_filme, F.nome_filme, F.genero_filme, F.imdb_link, F.img_link, F.popularidade_filme,
F.duracao_filme, F.sinopse, F.diretor_filme, F.ano_lancamento, F.nota_filme, F.produtora_filme,
GROUP_CONCAT(A.nome_ator) AS atores,
GROUP_CONCAT(I.nome_idioma) AS idiomas,
MAX(AV.nota_avaliacao) AS nota_avaliacao
FROM Usuario U
INNER JOIN Avaliacao AV ON U.id_usuario = AV.id_usuario
INNER JOIN Filme F ON AV.fk_filme_id = F.id_filme
LEFT JOIN filme_ator FA ON F.id_filme = FA.id_filme
LEFT JOIN Ator A ON FA.id_ator = A.id_ator
LEFT JOIN filme_idioma FI ON F.id_filme = FI.id_filme
LEFT JOIN idioma I ON FI.id_idioma = I.id_idioma
GROUP BY U.id_usuario, F.id_filme
'''

df = pd.read_sql_query(query, conn)

print(df)

    id_usuario username  id_filme   
0            1    user1         1  \
1            2    user2         2   
2            3    user3         3   
3            4    user4         4   
4            5    user5         5   
5            6    user6         6   
6            7    user7         7   
7            8    user8         8   
8            9    user9         9   
9           10   user10        10   
10          11   user11        11   
11          12   user12        12   
12          13   user13        13   
13          14   user14        14   
14          15   user15        15   
15          16   user16        16   
16          17   user17        17   
17          18   user18        18   
18          19   user19        19   
19          20   user20        20   

                                           nome_filme genero_filme   
0                            The Shawshank Redemption        Drama  \
1                                       The Godfather        Crime   
2           

Filmes lançados em 1999

In [27]:
query1 = "SELECT * FROM Filme WHERE ano_lancamento = 1999"
df1 = pd.read_sql_query(query1, conn)
print(df1)

   id_filme  nome_filme genero_filme                              imdb_link   
0         5  Fight Club        Drama  https://www.imdb.com/title/tt0137523/  \
1         8  The Matrix       Action  https://www.imdb.com/title/tt0133093/   

                             img_link  popularidade_filme  duracao_filme   
0  https://www.example.com/image5.jpg                 8.8          139.0  \
1  https://www.example.com/image8.jpg                 8.7          136.0   

                                             sinopse   
0  An insomniac office worker and a devil-may-car...  \
1  When a beautiful stranger leads computer hacke...   

                     diretor_filme  ano_lancamento  nota_filme   
0                    David Fincher            1999         8.8  \
1  Lana Wachowski, Lilly Wachowski            1999         8.7   

    produtora_filme  
0  20th Century Fox  
1      Warner Bros.  


Filmes com nota maior ou igual a 8.0

In [28]:
query2 = "SELECT * FROM Filme WHERE nota_filme >= 8.0"
df2 = pd.read_sql_query(query2, conn)
print(df2)

    id_filme                                         nome_filme genero_filme   
0          1                           The Shawshank Redemption        Drama  \
1          2                                      The Godfather        Crime   
2          3                                    The Dark Knight       Action   
3          4                                       Pulp Fiction        Crime   
4          5                                         Fight Club        Drama   
5          6                                       Forrest Gump        Drama   
6          7                                          Inception       Action   
7          8                                         The Matrix       Action   
8          9  The Lord of the Rings: The Fellowship of the Ring    Adventure   
9         10                                       The Avengers       Action   
10        11                                       Interstellar    Adventure   
11        12                            

Filmes do gênero Ação

In [30]:
query3 = "SELECT * FROM Filme WHERE genero_filme = 'Action'"
df3 = pd.read_sql_query(query3, conn)
print(df3)

   id_filme       nome_filme genero_filme   
0         3  The Dark Knight       Action  \
1         7        Inception       Action   
2         8       The Matrix       Action   
3        10     The Avengers       Action   
4        12        Gladiator       Action   

                               imdb_link                             img_link   
0  https://www.imdb.com/title/tt0468569/   https://www.example.com/image3.jpg  \
1  https://www.imdb.com/title/tt1375666/   https://www.example.com/image7.jpg   
2  https://www.imdb.com/title/tt0133093/   https://www.example.com/image8.jpg   
3  https://www.imdb.com/title/tt0848228/  https://www.example.com/image10.jpg   
4  https://www.imdb.com/title/tt0172495/  https://www.example.com/image12.jpg   

   popularidade_filme  duracao_filme   
0                 9.0          152.0  \
1                 8.7          148.0   
2                 8.7          136.0   
3                 8.6          143.0   
4                 8.5          155.0   

 

In [ ]:
conn.close()